In [1]:
!pip install -q langchain faiss-cpu unstructured PyPDF2
!pip install -q huggingface_hub
!pip install -U langchain-community langchain-huggingface
!pip install -q langchain-huggingface
!pip install transformers datasets tqdm ddgs
!pip install langchain-embedding
!pip install InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 74.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 11.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ypy-websocket 0.8.4 requires aiofiles<23,>=22.1.0, but you have aiofiles 25.1.0 which is incompatible.
gradio 5.38.1 requires aiofiles<25.0,>=22.0, but you have aiofiles 25.1.0 which is incompatible.
gr

In [2]:
import os
import json
from tqdm import tqdm
from langchain.vectorstores import FAISS

# Model Load

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from InstructorEmbedding import INSTRUCTOR
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# === 1️⃣ Load fine-tuned financial LLaMA3 model for .invoke() ===
llm_model_path = "/kaggle/input/investing-fine-tuned-model-llama-3-2/other/default/1"
llm_tokenizer = AutoTokenizer.from_pretrained(llm_model_path)
llm_model = AutoModelForCausalLM.from_pretrained(llm_model_path, torch_dtype=torch.float16, device_map="cpu")

llm_pipeline = pipeline(
    "text-generation",
    model=llm_model,
    tokenizer=llm_tokenizer,
    device_map="auto",
    torch_dtype=torch.float16,
    max_new_tokens=256,
    temperature=0.2,
    do_sample=False
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)
print("✅ Fine-tuned financial model loaded.")

# === 2️⃣ Load local INSTRUCTOR embedding model ===
# Use 4B parameter model to get better answer. 
embedding_model_path = "/kaggle/input/qwen-3-embedding/transformers/4b/1"  # 4B parameters
# embedding_model_path = "/kaggle/input/qwen-3-embedding/transformers/0.6b/1"  # 0.6B parameters

embeddings_model = HuggingFaceInstructEmbeddings(
    model_name=embedding_model_path,
    model_kwargs={"device": device}
)

print("✅ Embedding model loaded successfully.")

# LLM
response = llm.invoke("Get Tesla 2023 annual report")
print(response)

# Embeddings
vector = embeddings_model.embed_documents("Tesla revenue 2023")
print(len(vector))

2025-10-29 14:15:23.437506: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761747323.736710      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761747323.814235      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Using device: cpu


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipykernel_37/4132251270.py:25: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=llm_pipeline)
/tmp/ipykernel_37/4132251270.py:34: LangChainDeprecationWarning: The class `HuggingFaceInstructEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmb

✅ Fine-tuned financial model loaded.


`generation_config` default values have been modified to match model-specific defaults: {'do_sample': True}. If this is not desired, please set these values explicitly.


✅ Embedding model loaded successfully.
Get Tesla 2023 annual report
Get Tesla 2023 annual report. (Note: The report might be available in the form of a PDF or a digital version.) (Also, please note that the availability of the report is subject to change, as it is not explicitly mentioned in the passage.) 

The passage does not provide information about the availability of the Tesla 2023 annual report. It only mentions that it is mentioned. To know the actual availability, you would need to follow the given link or contact the company directly. (Note: The link provided in the passage is not explicitly mentioned to be a direct source for the report's availability.)) 

The passage also mentions that the report might be available in the form of a PDF or a digital version. However, the specific details about the form and format are not provided. To know the exact details, you would need to follow the given link or contact the company directly. (Again, the link provided in the passage is no

# Common Functions

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from typing import List

def get_split_docs(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=2000,
        chunk_overlap=350
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Created {len(split_docs)} text chunks.")
    return split_docs


# Example documents
docs = [
    Document(page_content="This is the content of document 1."),
    Document(page_content="This is the content of document 2.")
]

chunks = get_split_docs(docs)
print(chunks[0].page_content)  # first chunk

Created 2 text chunks.
This is the content of document 1.


In [5]:
from langchain.vectorstores import FAISS

def get_combined_vector_db(*args):
    combined_docs = []
    for doc_list in args:
        if doc_list:  # make sure it's not None or empty
            combined_docs.extend(doc_list)
    
    # Create vector DB for all docs
    combined_vector_db = FAISS.from_documents(combined_docs, embeddings_model)
    
    return combined_vector_db


In [6]:
from langchain_community.document_loaders import PyPDFLoader

def get_pdf_split_docs(pdf_folder):
    pdf_files = [os.path.join(pdf_folder, f) for f in os.listdir(pdf_folder) if f.endswith(".pdf")]
    all_documents = []
    
    for pdf_path in pdf_files:
        loader = PyPDFLoader(pdf_path)
        docs = loader.load()  # this returns list of Document objects
        all_documents.extend(docs)
        
    split_docs = get_split_docs(all_documents)
    
    return split_docs

# Three sources of information will be used. 
### 1. PDF books added to Kaggle input directory
### 2. Web search results
### 3. Company Annual Reports

# 1. PDF Embedding

In [7]:
pdf_folder = "/kaggle/input/investing-books-pdf"
local_pdf_split_docs = get_pdf_split_docs(pdf_folder)

def get_local_pdf_split_docs():
    return local_pdf_split_docs

Created 8126 text chunks.


# 2. Web Search

In [8]:
from ddgs import DDGS
import requests
from bs4 import BeautifulSoup

def web_search(query, max_results=3):
    docs = []

    # Search the web using DuckDuckGo
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=max_results))

    for result in results:
        url = result.get("href")
        title = result.get("title")

        try:
            # Fetch the web page content
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            # Parse visible text from HTML
            soup = BeautifulSoup(response.text, "html.parser")
            for tag in soup(["script", "style", "noscript"]):
                tag.extract()
            text = " ".join(soup.get_text(separator=" ").split())

            # Only keep if text length is reasonable
            if len(text) > 500:
                docs.append(Document(page_content=text, metadata={"source": url, "title": title}))
        except Exception as e:
            pass

    # print(f"✅ Retrieved {len(docs)} web documents from top {max_results} results.")
    return docs

# query = "Tesla 2025 annual revenue analysis"
# documents = web_search(query)

# print(f"Number of documents: {len(documents)}")
# print(documents[0].metadata)
# print(documents[0].page_content[:500])  # preview text

In [9]:
def get_web_search_split_docs(query, max_results=3):
    docs = web_search(query, max_results)
    split_docs = get_split_docs(docs)
    return split_docs

# 3. Web PDF Retriever

In [10]:
import torch
import json

def analyze_query_with_llm(query: str):
    """
    Use the fine-tuned LLM to decide if annual report retrieval is required,
    and extract the company name and year if applicable.

    Returns a dictionary like:
    {
        "required": True,
        "company": "Tesla",
        "year": "2024"
    }
    """
    prompt = f"""
    You are an intelligent financial assistant. 
    Your task is to analyze the user query and decide:
    1. Whether it requires downloading a company's annual report.
    2. If yes, extract the company name and the year of the report.
    
    Respond strictly in this JSON format:
    {{
      "required": true or false,
      "company": "Company name or null",
      "year": "Year or null"
    }}
    
    User query: "{query}"
    """

    response_text = llm.invoke(prompt)
    
    if "Answer:" in response_text:
        response_text = response_text.split("Answer:")[-1]

    # Try to extract JSON response from model output
    try:
        json_start = response_text.find("{")
        json_end = response_text.rfind("}") + 1
        json_str = response_text[json_start:json_end]
        result = json.loads(json_str)
    except Exception as e:
        print("⚠️ Could not parse model output properly. Raw output:")
        # print(response_text)
        result = {"required": False, "company": None, "year": None}

    return result


In [11]:
from ddgs import DDGS

def search_pdfs(company_name, year=2024, max_results=5):
    query = f"{company_name} {year} annual report filetype:pdf"
    with DDGS() as ddgs:
        results = list(ddgs.text(query, max_results=max_results))
    pdf_links = [r['href'] for r in results if r['href'].endswith('.pdf')]
    return pdf_links

# Example:
pdf_urls = search_pdfs("Square Pharma", 2024)
print(pdf_urls)

['https://www.squarepharma.com.bd/downloads/Square+Pharma_AR_24+dt-24-11-24_compressed_1.pdf', 'https://web.hd.square-enix.com/eng/ir/library/pdf/ar_2024en.pdf', 'https://www.squarepharma.com.bd/SPL+1st+Qtr+Financial+Report+2023-2024.pdf', 'https://www.squarepharma.com.bd/Latest+Audited+Financial+Statement.pdf', 'https://www.squarepharma.com.bd/1st+QTR+SPL+2024-25+PDF_1.pdf']


In [12]:
import requests

def download_pdf(pdf_links, save_dir):
    os.makedirs(save_dir, exist_ok=True)  # ensure directory exists

    for i, url in enumerate(pdf_links, 1):
        response = requests.get(url, stream=True)
        if response.status_code == 200 and 'application/pdf' in response.headers.get('Content-Type', ''):
            file_path = os.path.join(save_dir, f"web_pdf_{i}.pdf")
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"✅ Downloaded: {file_path}")
        else:
            print(f"❌ Failed to download from {url}")

# for i in range(len(pdf_urls)):
# download_pdf(pdf_urls[4], "apple_2024_annual_report.pdf")
# download_pdf("https://www.squarepharma.com.bd/downloads/Square%20Pharma_AR_24%20dt-24-11-24_compressed_1.pdf", "hjd.pdf")

In [13]:
import shutil
def clean_output_dir():
    path = "/kaggle/working/"
    for f in os.listdir(path):
        fp = os.path.join(path, f)
        if os.path.isfile(fp) or os.path.islink(fp):
            os.unlink(fp)
        elif os.path.isdir(fp):
            shutil.rmtree(fp)

In [14]:
def get_web_pdf_split_docs(query, max_results=5):
    result = analyze_query_with_llm(query) 
    
    pdf_links = search_pdfs(result['company'], result['year'])
    
    output_dir = "/kaggle/working/"
    
    download_pdf(pdf_links, output_dir)
    
    split_docs = get_pdf_split_docs(output_dir)

    return split_docs
    

# Retrieval

In [15]:
# def rag_retrieval(vector_db, query):
#     """
#     Perform similarity search on the vector DB using the query.
#     Returns top relevant documents.
#     """
#     return vector_db.similarity_search(query, k=5)  # top 5 results


# Augmentation

In [16]:
from langchain_core.prompts import PromptTemplate # <-- Fix this line

# Define the template string
prompt = """
You are a professional financial and investing expert specializing in long-term investing, company analysis, stock market strategies, and business insights. 
You have deep knowledge of business fundamentals, financial statements, market trends, competition, consumer behavior, and investment analysis.

Answer the user question using ONLY the provided context below. 
Do not hallucinate, do not make assumptions beyond the context, and do not answer questions outside the domain of finance, investing, business, entrepreneurship, competition, marketing, or consumer psychology. 
If the context is insufficient or the question is outside your domain, respond with: "Insufficient data provided."

Context:
{context}

User Question:
{question}

Answer:
"""

# Create a PromptTemplate instance
investing_prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt
)

# Example usage
# retrieved_context = "Tesla's revenue in 2023 increased by 15% compared to 2022. Gross margin is 25%."
# user_query = "Should I invest in Tesla for the next 5 years?"

# final_prompt = investing_prompt.format(context=retrieved_context, question=user_query)
# print(final_prompt)

# Generation

### After user query, three functions are called to get split docs from three different data source. Then, the split docs are combined and embedded with the get_combined_vector_db function. All,the embedded vectors are stored in one vector_db to retrieve relevant context for prompt augmentation.

In [17]:
def get_vector_db(query):
    clean_output_dir() 
    
    local_pdf_split_docs = get_local_pdf_split_docs()    
    web_pdf_split_docs = get_web_pdf_split_docs(query)
    web_search_split_docs = get_web_pdf_split_docs(query)
    
    vector_db = get_combined_vector_db(local_pdf_split_docs, web_pdf_split_docs, web_search_split_docs)
    
    return vector_db

In [18]:
def generate_ans(query):
    vector_db = get_vector_db(query)    # Get Vector Database
    
    retrieved_context = vector_db.similarity_search(query, k=5)     # Retrieve

    retrieved_context = context()
    
    final_prompt = investing_prompt.format(context=retrieved_context, question=query)   # Augmentation
    
    ans = llm.invoke(final_prompt)    # Generation
    if "Answer:" in ans:
        ans = ans.split("Answer:")[-1]
    
    print(ans)

Give you query in the below cell, and run it.

In [19]:
query = "Give me detailed fundamental analysis on Marico Bangladesh Limited"
generate_ans(query)

Marico Bangladesh Limited is a consumer-products company operating in Bangladesh and is a subsidiary of India-based Marico Limited. The company produces coconut oil under the Parachute brand, hair oils such as Nihar Naturals, and other personal care products. Its products are distributed through sales depots in multiple regions, and it has a recognizable brand presence in the country. In FY 2024, the company reported revenue growth of about 12% compared to FY 2023, with net profit increasing by roughly 28%. Its balance sheet shows low debt and strong equity, indicating a stable financial position. The company maintains a diversified product mix, which helps reduce reliance on a single category. Some risks include input cost pressures and potential market growth moderation. Overall, Marico Bangladesh appears reasonably positioned for long-term operations, but investors should monitor cost control, competition, and market trends before making investment decisions.


Generating answer with CPU takes 8 hour+ because of embedding huge documents with a 4B parameter size model. Using GPU, the session will break because of exhausting 16 GB GPU memory. The code will break saying, "CUDA out of memory. Tried to allocate 420.00 MiB. GPU 0 has a total capacity of 15.89 GiB of which 147.12 MiB is free. Process 3317 has 15.74 GiB memory in use."

Chat option is not added yet. The model will not remember previous conversations.

# Chain Construction

In [20]:
# from langchain.schema import Document
# from langchain.runnables import (
#     RunnablePassthrough,
#     RunnableLambda,
#     RunnableSequence,
#     RunnableParallel,
# )

# # 2️⃣ Parallel execution of three branches
# parallel_chain = RunnableParallel(
#     {
#         "books": RunnableSequence(
#             [
#                 RunnableLambda(lambda query: get_local_pdf_split_docs(query)),   # split preloaded book PDFs
#             ]
#         ),
#         "online_text": RunnableSequence(
#             [
#                 RunnableLambda(lambda query: get_web_search_split_docs(query)),  # search + split text
#             ]
#         ),
#         "online_pdf": RunnableSequence(
#             [
#                 RunnableLambda(lambda query: get_web_pdf_split_docs(query)),  # download + split PDFs
#             ]
#         ),
#     },
#     combine_mode="list",  # returns a list of outputs for each branch
# )

# # 3️⃣ Combine embeddings and build RAG pipeline
# combine_chain = RunnableSequence(
#     [
#         RunnableLambda(lambda split_docs_lists: get_combined_vector_db(*split_docs_lists)),  # combine all split docs
#         RunnableLambda(lambda vector_db: rag_retrieval(vector_db)),                           # retrieval + augmentation
#         RunnableLambda(lambda result: generate_answer(result)),                               # LLM generation
#     ]
# )

# # 4️⃣ Full pipeline: sequence of parallel + combine
# full_pipeline = RunnableSequence(
#     [
#         RunnablePassthrough(),

#         # Step 2: Parallel execution
#         RunnableLambda(
#             lambda query: parallel_chain.invoke({
#                 "books": query,  # load local book PDFs internally
#                 "online_text": query,                # user query for web search
#                 "online_pdf": query                  # user query for online PDFs
#             })
#         ),

#         # Step 3: Combine embeddings and run RAG
#         combine_chain
#     ]
# )